In [ ]:
import os
import re
import sys
from pathlib import Path

def rename_images(root_dir):
    for sub_dir in os.scandir(root_dir):
        if sub_dir.is_dir():
            for time_dir in os.scandir(sub_dir.path):
                if time_dir.is_dir():
                    time_folder_name = time_dir.name
                    clean_time = re.sub(r'\D', '', time_folder_name)
                    for img_file in os.scandir(time_dir.path):
                        if img_file.is_file() and img_file.name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                            base_name = os.path.basename(img_file.name)
                            name_without_ext = os.path.splitext(base_name)[0]
                            match = re.search(r'\d{5}$', name_without_ext)
                            if match:
                                last_five_digits = match.group(0)
                            else:
                                digits = re.findall(r'\d+', name_without_ext)
                                if digits:
                                    last_digits = digits[-1]
                                    last_five_digits = last_digits.zfill(5)[-5:]
                                else:
                                    import random
                                    last_five_digits = f"{random.randint(0, 99999):05d}"
                                    print(f"警告: 无法从 {base_name} 中提取数字，使用随机数字: {last_five_digits}")
                            new_name = f"{clean_time}-{last_five_digits}{os.path.splitext(base_name)[1]}"
                            new_path = os.path.join(time_dir.path, new_name)
                            try:
                                os.rename(img_file.path, new_path)
                                print(f"已重命名: {img_file.path} -> {new_path}")
                            except Exception as e:
                                print(f"错误: 无法重命名 {img_file.path}: {e}")

def get_directory_path():
    while True:
        path = input("请输入待处理文件夹的路径 (输入q退出): ").strip()      
        if path.lower() == 'q':
            sys.exit(0)        
        if os.path.isdir(path):
            return path
        else:
            print(f"错误: '{path}' 不是一个有效的目录路径，请重新输入。")
if __name__ == "__main__":
    print("=== 批量图片重命名工具 ===")
    root_directory = get_directory_path()
    print(f"开始处理目录: {root_directory}")
    rename_images(root_directory)
    print("处理完成!")    

=== 批量图片重命名工具 ===
开始处理目录: E:\Desktop\non-stable
已重命名: E:\Desktop\non-stable\001\2024-12-12 16,25,30\DSC00188.JPG -> E:\Desktop\non-stable\001\2024-12-12 16,25,30\20241212162530-00188.JPG
已重命名: E:\Desktop\non-stable\001\2024-12-12 16,25,30\DSC00189.JPG -> E:\Desktop\non-stable\001\2024-12-12 16,25,30\20241212162530-00189.JPG
已重命名: E:\Desktop\non-stable\001\2024-12-12 16,25,30\DSC00190.JPG -> E:\Desktop\non-stable\001\2024-12-12 16,25,30\20241212162530-00190.JPG
已重命名: E:\Desktop\non-stable\001\2024-12-12 16,25,30\DSC00191.JPG -> E:\Desktop\non-stable\001\2024-12-12 16,25,30\20241212162530-00191.JPG
已重命名: E:\Desktop\non-stable\002\2024-12-26 15,32,58\DSC00183.JPG -> E:\Desktop\non-stable\002\2024-12-26 15,32,58\20241226153258-00183.JPG
已重命名: E:\Desktop\non-stable\002\2024-12-26 15,32,58\DSC00184.JPG -> E:\Desktop\non-stable\002\2024-12-26 15,32,58\20241226153258-00184.JPG
已重命名: E:\Desktop\non-stable\002\2024-12-26 15,32,58\DSC00185.JPG -> E:\Desktop\non-stable\002\2024-12-26 15,32,58\2024

In [ ]:
import os
import re
import sys
from pathlib import Path

def get_wood_lamp_images(remarks_path):
    """从remarks.txt中提取伍德灯图片名称（精确匹配后段字符串）"""
    wood_lamp_images = set()
    wood_pattern = r"PMRGE33EPFPXAYLSOQRDUITXN5XWJZ4BV7SLRCZCFQRGS3LBM5SV64TFNVQXE2ZCHIRHO33PMTTYDL7EXCFSE7I$"   
    try:
        with open(remarks_path, 'r', encoding='utf-8-sig') as f:
            for line in f:
                line = line.strip()
                if '=' in line:
                    img_name, code = line.split('=', 1)
                    img_name = img_name.strip()
                    code = code.strip()
                    if re.fullmatch(wood_pattern, code):
                        wood_lamp_images.add(img_name)
                        print(f"匹配到伍德灯图片: {img_name}")
    except Exception as e:
        print(f"读取remarks文件出错: {e}")
    return wood_lamp_images
def process_images(root_dir):
    """处理所有子文件夹中的图片并按规则重命名"""
    global_counter = 1
    for seq_dir in sorted(os.scandir(root_dir), key=lambda x: int(x.name)):
        if not seq_dir.is_dir():
            continue   
        print(f"\n处理子文件夹: {seq_dir.name}")
        time_dirs = [d for d in os.scandir(seq_dir.path) if d.is_dir()]
        if not time_dirs:
            print(f"  警告: 无时间戳文件夹，跳过")
            continue 
        for time_dir in sorted(time_dirs, key=lambda x: x.name):
            time_folder_name = time_dir.name
            clean_time = re.sub(r'\D', '', time_folder_name)
            if not clean_time:
                print(f"  警告: 时间戳文件夹名称 '{time_folder_name}' 无数字，跳过")
                continue       
            print(f"  处理时间戳文件夹: {time_folder_name}")
            remarks_path = os.path.join(time_dir.path, 'remarks.txt')
            if not os.path.exists(remarks_path):
                print(f"  警告: 未找到remarks.txt，跳过")
                continue
            wood_lamp_images = get_wood_lamp_images(remarks_path)
            if not wood_lamp_images:
                print(f"  警告: 无伍德灯图片信息，跳过")
                continue
            image_files = []
            for entry in os.scandir(time_dir.path):
                if entry.is_file() and entry.name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    image_files.append(entry)   
            if not image_files:
                print(f"  警告: 无图片文件，跳过")
                continue
            wood_files = []
            normal_files = []
            for img in image_files:
                img_name = img.name
                if img_name in wood_lamp_images:
                    wood_files.append(img)
                    print(f"  伍德灯图片: {img_name}")
                else:
                    normal_files.append(img)
                    print(f"  普通图片: {img_name}")
            wood_files.sort(key=lambda x: x.name)
            normal_files.sort(key=lambda x: x.name)
            wood_count = len(wood_files)
            normal_count = len(normal_files)
            print(f"  伍德灯图片数: {wood_count}, 普通图片数: {normal_count}")
            if wood_count == 0 or normal_count == 0:
                print(f"  警告: 伍德灯或普通图片数为0，无法配对")
                continue
            paired_count = 0
            for i in range(min(wood_count, normal_count)):
                wood_img = wood_files[i]
                normal_img = normal_files[i]
                pair_num = f"{global_counter:04d}"
                wood_suffix = re.search(r'(\d{5})(?=\.\w+$)', wood_img.name) or \
                             re.search(r'(\d{3,})(?=\.\w+$)', wood_img.name)
                normal_suffix = re.search(r'(\d{5})(?=\.\w+$)', normal_img.name) or \
                               re.search(r'(\d{3,})(?=\.\w+$)', normal_img.name)
                wood_suffix_num = wood_suffix.group(1)[-5:].zfill(5) if wood_suffix else "00000"
                normal_suffix_num = normal_suffix.group(1)[-5:].zfill(5) if normal_suffix else "00000"
                wood_new_name = f"00-{pair_num}-{clean_time}-{wood_suffix_num}{os.path.splitext(wood_img.name)[1]}"
                normal_new_name = f"01-{pair_num}-{clean_time}-{normal_suffix_num}{os.path.splitext(normal_img.name)[1]}"
                try:
                    os.rename(wood_img.path, os.path.join(time_dir.path, wood_new_name))
                    os.rename(normal_img.path, os.path.join(time_dir.path, normal_new_name))
                    print(f"  ✅ 成功配对重命名:")
                    print(f"    {wood_img.name} -> {wood_new_name}")
                    print(f"    {normal_img.name} -> {normal_new_name}")
                    paired_count += 1
                    global_counter += 1
                except Exception as e:
                    print(f"  ❌ 重命名失败: {e}") 
            print(f"  时间戳文件夹处理完成: {paired_count} 对，当前全局序号: {global_counter-1}")  
    print(f"\n所有文件夹处理完成，共生成 {global_counter-1} 个配对序号")
def get_root_directory():
    """获取并验证根目录路径"""
    while True:
        path = input("请输入根目录路径 (输入q退出): ").strip()
        if path.lower() == 'q':
            sys.exit(0)
        if os.path.isdir(path):
            return path
        print(f"错误: '{path}' 不是有效目录，请重新输入")
if __name__ == "__main__":
    print("=== 伍德灯图片配对重命名工具 (修复版) ===")
    root_dir = get_root_directory()
    process_images(root_dir)
    print("处理完成!")

=== 伍德灯图片配对重命名工具 (修复版) ===

处理子文件夹: 001
  处理时间戳文件夹: 2024-12-12 16,25,30
匹配到伍德灯图片: DSC00189.JPG
匹配到伍德灯图片: DSC00191.JPG
  普通图片: DSC00188.JPG
  伍德灯图片: DSC00189.JPG
  普通图片: DSC00190.JPG
  伍德灯图片: DSC00191.JPG
  伍德灯图片数: 2, 普通图片数: 2
  ✅ 成功配对重命名:
    DSC00189.JPG -> 00-0001-20241212162530-00189.JPG
    DSC00188.JPG -> 01-0001-20241212162530-00188.JPG
  ✅ 成功配对重命名:
    DSC00191.JPG -> 00-0002-20241212162530-00191.JPG
    DSC00190.JPG -> 01-0002-20241212162530-00190.JPG
  时间戳文件夹处理完成: 2 对，当前全局序号: 2

处理子文件夹: 002
  处理时间戳文件夹: 2024-12-26 15,32,58
匹配到伍德灯图片: DSC00185.JPG
匹配到伍德灯图片: DSC00187.JPG
  普通图片: DSC00183.JPG
  普通图片: DSC00184.JPG
  伍德灯图片: DSC00185.JPG
  伍德灯图片: DSC00187.JPG
  伍德灯图片数: 2, 普通图片数: 2
  ✅ 成功配对重命名:
    DSC00185.JPG -> 00-0003-20241226153258-00185.JPG
    DSC00183.JPG -> 01-0003-20241226153258-00183.JPG
  ✅ 成功配对重命名:
    DSC00187.JPG -> 00-0004-20241226153258-00187.JPG
    DSC00184.JPG -> 01-0004-20241226153258-00184.JPG
  时间戳文件夹处理完成: 2 对，当前全局序号: 4

处理子文件夹: 003
  处理时间戳文件夹: 2025-04-07 14,56,2

In [ ]:
import os
import re
import sys
from pathlib import Path
def get_wood_lamp_images_from_remarks(remarks_path):
    wood_lamp_images = set()
    try:
        with open(remarks_path, 'r', encoding='utf-8-sig') as f:
            for line in f:
                line = line.strip()
                if '=' in line and line.endswith('7EXCFSE7I'):
                    img_name, _ = line.split('=', 1)
                    img_name = img_name.strip()
                    wood_lamp_images.add(img_name)
                    print(f"匹配的伍德灯图片行: {line}")
        return wood_lamp_images
    except Exception as e:
        print(f"错误: 无法读取remarks文件 {remarks_path}: {e}")
        return set()

def process_images(root_dir):
    global_counter = 1
    for seq_dir in sorted(os.scandir(root_dir), key=lambda x: x.name):
        if not seq_dir.is_dir():
            continue      
        print(f"\n处理子文件夹: {seq_dir.path}")
        time_dirs = [d for d in os.scandir(seq_dir.path) if d.is_dir()]
        if not time_dirs:
            print(f"  警告: 子文件夹 {seq_dir.path} 中没有时间戳文件夹，跳过")
            continue     
        for time_dir in sorted(time_dirs, key=lambda x: x.name):
            remarks_path = os.path.join(time_dir.path, 'remarks.txt')
            if not os.path.exists(remarks_path):
                print(f"  警告: 在时间文件夹 {time_dir.path} 中未找到remarks.txt文件，跳过")
                continue                
            print(f"  处理时间文件夹: {time_dir.path}")
            wood_lamp_images = get_wood_lamp_images_from_remarks(remarks_path)
            if not wood_lamp_images:
                print(f"    警告: 在remarks.txt中未找到伍德灯图片信息，跳过")
                continue
            time_folder_name = time_dir.name
            clean_time = re.sub(r'\D', '', time_folder_name)
            if not clean_time:
                print(f"    警告: 无法从时间文件夹名称 '{time_folder_name}' 中提取数字，跳过")
                continue
            image_files = []
            for entry in os.scandir(time_dir.path):
                if entry.is_file() and entry.name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                    image_files.append(entry.path)           
            if not image_files:
                print(f"    警告: 时间文件夹 {time_dir.path} 中没有图片，跳过")
                continue
            wood_lamp_files = []
            normal_files = []    
            for img_path in image_files:
                img_name = os.path.basename(img_path)
                if img_name in wood_lamp_images:
                    wood_lamp_files.append(img_path)
                else:
                    normal_files.append(img_path)
            wood_lamp_files.sort(key=lambda x: os.path.basename(x))
            normal_files.sort(key=lambda x: os.path.basename(x))
            paired_count = 0
            unpaired_wood_lamp = []
            unpaired_normal = []
            min_len = min(len(wood_lamp_files), len(normal_files))
            for i in range(min_len):
                wood_path = wood_lamp_files[i]
                normal_path = normal_files[i]
                pair_number = f"{global_counter:04d}"
                wood_basename = os.path.basename(wood_path)
                wood_ext = os.path.splitext(wood_basename)[1]
                wood_new_name = f"10-{pair_number}-{clean_time}-{wood_basename[-9:-4]}{wood_ext}"
                normal_basename = os.path.basename(normal_path)
                normal_ext = os.path.splitext(normal_basename)[1]
                normal_new_name = f"11-{pair_number}-{clean_time}-{normal_basename[-9:-4]}{normal_ext}"
                try:
                    os.rename(wood_path, os.path.join(time_dir.path, wood_new_name))
                    os.rename(normal_path, os.path.join(time_dir.path, normal_new_name))
                    print(f"    已配对重命名: {wood_basename} -> {wood_new_name} | {normal_basename} -> {normal_new_name}")
                    global_counter += 1
                    paired_count += 1
                except Exception as e:
                    print(f"    错误: 无法重命名 {wood_path} 或 {normal_path}: {e}")
            if len(wood_lamp_files) > len(normal_files):
                unpaired_wood_lamp = wood_lamp_files[min_len:]
            if len(normal_files) > len(wood_lamp_files):
                unpaired_normal = normal_files[min_len:]
            for img_path in unpaired_wood_lamp + unpaired_normal:
                img_basename = os.path.basename(img_path)
                img_ext = os.path.splitext(img_basename)[1]
                prefix = "10" if img_path in wood_lamp_files else "11"
                pair_number = f"{global_counter:04d}"
                new_name = f"{prefix}-{pair_number}-{clean_time}-{img_basename[-9:-4]}{img_ext}"
                try:
                    os.rename(img_path, os.path.join(time_dir.path, new_name))
                    print(f"    已单独重命名: {img_basename} -> {new_name}")
                    global_counter += 1
                except Exception as e:
                    print(f"    错误: 无法重命名 {img_path}: {e}")           
            print(f"    时间文件夹 {time_dir.path} 处理完成: {paired_count} 对，{len(unpaired_wood_lamp + unpaired_normal)} 张单独图片") 
    print(f"\n所有文件夹处理完成，共生成 {global_counter-1} 个序号")
def get_directory_path():
    """获取并验证用户输入的目录路径"""
    while True:
        path = input("请输入待处理文件夹的路径 (输入q退出): ").strip()      
        if path.lower() == 'q':
            sys.exit(0)      
        if os.path.isdir(path):
            return path
        else:
            print(f"错误: '{path}' 不是一个有效的目录路径，请重新输入。")
if __name__ == "__main__":
    print("=== 伍德灯和普通图片配对重命名工具 ===")
    root_directory = get_directory_path()   
    process_images(root_directory)  
    print("\n所有处理完成!")

=== 伍德灯和普通图片配对重命名工具 ===

处理子文件夹: E:\Desktop\non-stable\001
  处理时间文件夹: E:\Desktop\non-stable\001\2024-12-12 16,25,30
匹配的伍德灯图片行: DSC00189.JPG=PMRGE33EPFPXAYLSOQRDUITXN5XWJZ4BV7SLRCZCFQRGS3LBM5SV64TFNVQXE2ZCHIRHO33PMTTYDL7EXCFSE7I
匹配的伍德灯图片行: DSC00191.JPG=PMRGE33EPFPXAYLSOQRDUITXN5XWJZ4BV7SLRCZCFQRGS3LBM5SV64TFNVQXE2ZCHIRHO33PMTTYDL7EXCFSE7I
    已单独重命名: 20241212162530-00188.JPG -> 11-0001-20241212162530-00188.JPG
    已单独重命名: 20241212162530-00189.JPG -> 11-0002-20241212162530-00189.JPG
    已单独重命名: 20241212162530-00190.JPG -> 11-0003-20241212162530-00190.JPG
    已单独重命名: 20241212162530-00191.JPG -> 11-0004-20241212162530-00191.JPG
    时间文件夹 E:\Desktop\non-stable\001\2024-12-12 16,25,30 处理完成: 0 对，4 张单独图片

处理子文件夹: E:\Desktop\non-stable\002
  处理时间文件夹: E:\Desktop\non-stable\002\2024-12-26 15,32,58
匹配的伍德灯图片行: DSC00185.JPG=PMRGE33EPFPXAYLSOQRDUITXN5XWJZ4BV7SLRCZCFQRGS3LBM5SV64TFNVQXE2ZCHIRHO33PMTTYDL7EXCFSE7I
匹配的伍德灯图片行: DSC00187.JPG=PMRGE33EPFPXAYLSOQRDUITXN5XWJZ4BV7SLRCZCFQRGS3LBM5SV64TFNVQXE2ZCHI

In [ ]:
import os
import shutil
from pathlib import Path

def extract_jpg_images(root_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    copied_count = 0
    skipped_count = 0
    for dirpath, _, filenames in os.walk(root_dir):
        if os.path.abspath(dirpath) == os.path.abspath(target_dir):
            continue       
        print(f"处理文件夹: {dirpath}")
        for filename in filenames:
            if filename.lower().endswith(('.jpg', '.jpeg')):
                try:
                    file_path = os.path.join(dirpath, filename)
                    target_file_path = os.path.join(target_dir, filename)
                    if os.path.exists(target_file_path):
                        base_name, ext = os.path.splitext(filename)
                        counter = 1
                        while True:
                            new_filename = f"{base_name}_{counter}{ext}"
                            new_target_path = os.path.join(target_dir, new_filename)
                            if not os.path.exists(new_target_path):
                                target_file_path = new_target_path
                                break
                            counter += 1
                    shutil.copy2(file_path, target_file_path)
                    copied_count += 1
                    print(f"  已复制: {filename} → {os.path.basename(target_file_path)}")
                except Exception as e:
                    print(f"  错误: 无法复制 {filename}: {e}")
                    skipped_count += 1
    print("\n处理完成!")
    print(f"总共复制了 {copied_count} 个JPG文件")
    print(f"跳过了 {skipped_count} 个文件")
if __name__ == "__main__":
    root_directory = input("请输入根目录路径: ").strip()
    target_directory = 'non-stable_end'
    if not os.path.isdir(root_directory):
        print(f"错误: 目录 '{root_directory}' 不存在")
    else:
        extract_jpg_images(root_directory, target_directory)

处理文件夹: E:\Desktop\non_stable
处理文件夹: E:\Desktop\non_stable\001
处理文件夹: E:\Desktop\non_stable\001\2024-12-12 16,25,30
  已复制: 00-0001-20241212162530-00189.JPG → 00-0001-20241212162530-00189.JPG
  已复制: 00-0002-20241212162530-00191.JPG → 00-0002-20241212162530-00191.JPG
  已复制: 01-0001-20241212162530-00188.JPG → 01-0001-20241212162530-00188.JPG
  已复制: 01-0002-20241212162530-00190.JPG → 01-0002-20241212162530-00190.JPG
处理文件夹: E:\Desktop\non_stable\002
处理文件夹: E:\Desktop\non_stable\002\2024-12-26 15,32,58
  已复制: 00-0003-20241226153258-00185.JPG → 00-0003-20241226153258-00185.JPG
  已复制: 00-0004-20241226153258-00187.JPG → 00-0004-20241226153258-00187.JPG
  已复制: 01-0003-20241226153258-00184.JPG → 01-0003-20241226153258-00184.JPG
  已复制: 01-0004-20241226153258-00183.JPG → 01-0004-20241226153258-00183.JPG
处理文件夹: E:\Desktop\non_stable\003
处理文件夹: E:\Desktop\non_stable\003\2025-04-07 14,56,27
  已复制: 00-0005-20250407145627-00336.JPG → 00-0005-20250407145627-00336.JPG
  已复制: 01-0005-20250407145627-00335.JP

In [ ]:
import os
import re

def sort_and_rename_images(folder_path):
    pattern = r'^11-(\d{4})-(\d+)-(\d{5})\.(JPG|jpg)$'
    files = []
    for filename in os.listdir(folder_path):
        match = re.match(pattern, filename)
        if match:
            middle_digits = match.group(1)
            date_part = match.group(2)
            end_digits = match.group(3)
            extension = match.group(4)          
            files.append({
                'original_filename': filename,
                'middle_digits': middle_digits,
                'date_part': date_part,
                'end_digits': end_digits,
                'extension': extension
            })
    files.sort(key=lambda x: (x['middle_digits'], x['original_filename']))
    new_middle_digits = []
    for i in range(1, len(files) + 1):
        new_middle_digits.append(f"{i:04d}")
    for i, file in enumerate(files):
        old_path = os.path.join(folder_path, file['original_filename'])
        new_filename = f"11-{new_middle_digits[i]}-{file['date_part']}-{file['end_digits']}.{file['extension']}"
        new_path = os.path.join(folder_path, new_filename)     
        try:
            os.rename(old_path, new_path)
            print(f"已重命名: {file['original_filename']} -> {new_filename}")
        except Exception as e:
            print(f"重命名失败: {file['original_filename']} -> {new_filename}, 错误: {e}")
    
    print(f"成功重命名{len(files)}个文件")
folder_path = r"E:\Desktop\stable_clinic"  # 请替换为实际的文件夹路径
sort_and_rename_images(folder_path)

已重命名: 11-0001-20240725121302-00150.JPG -> 11-0001-20240725121302-00150.JPG
已重命名: 11-0002-20240827081753-00001.JPG -> 11-0002-20240827081753-00001.JPG
已重命名: 11-0003-20230202113446-00167.JPG -> 11-0003-20230202113446-00167.JPG
已重命名: 11-0004-20230202113446-00170.JPG -> 11-0004-20230202113446-00170.JPG
已重命名: 11-0005-20230202113446-00175.JPG -> 11-0005-20230202113446-00175.JPG
已重命名: 11-0006-20230202113446-00173.JPG -> 11-0006-20230202113446-00173.JPG
已重命名: 11-0007-20220207095056-00044.JPG -> 11-0007-20220207095056-00044.JPG
已重命名: 11-0008-20220222105317-00157.JPG -> 11-0008-20220222105317-00157.JPG
已重命名: 11-0009-20220222105317-00159.JPG -> 11-0009-20220222105317-00159.JPG
已重命名: 11-0010-20220222105317-00161.JPG -> 11-0010-20220222105317-00161.JPG
已重命名: 11-0010-20230109102144-00120.JPG -> 11-0011-20230109102144-00120.JPG
已重命名: 11-0011-20230109102144-00123.JPG -> 11-0012-20230109102144-00123.JPG
已重命名: 11-0012-20220322082233-00009.JPG -> 11-0013-20220322082233-00009.JPG
已重命名: 11-0013-20220322082